# REQUIRES LARGE/GPU INSTANCE TYPE
## google/flan-t5-base toxicity score:
Mean: 0.11131308024933918 - Std: 0.26920890184712915

In [10]:
import psutil

notebook_memory = psutil.virtual_memory()
print(notebook_memory)

if notebook_memory.total < 32 * 1000 * 1000 * 1000:
    print('*******************************************')    
    print('YOU ARE NOT USING THE CORRECT INSTANCE TYPE')
    print('PLEASE CHANGE INSTANCE TYPE TO  m5.2xlarge ')
    print('*******************************************')
else:
    correct_instance_type=True

svmem(total=66814242816, available=64678019072, percent=3.2, used=1444524032, free=53346021376, active=1792905216, inactive=10226774016, buffers=2768896, cached=12020928512, shared=905216, slab=856301568)


In [11]:
%store -r pretrained_model_checkpoint

In [12]:
try:
    peft_lora_fine_tuned_model_checkpoint
except NameError:
    print("++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++")
    print("[ERROR] Please run the notebooks in the PREPARE section before you continue.")
    print("++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++")

++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++
[ERROR] Please run the notebooks in the PREPARE section before you continue.
++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++


In [13]:
print(pretrained_model_checkpoint)

t5-base


In [14]:
%store -r peft_lora_fine_tuned_model_checkpoint

In [15]:
try:
    peft_lora_fine_tuned_model_checkpoint
except NameError:
    print("++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++")
    print("[ERROR] Please run the notebooks in the PREPARE section before you continue.")
    print("++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++")

In [16]:
print(peft_lora_fine_tuned_model_checkpoint)

./peft-lora-fine-tune-model-checkpoint


# Quantitative Results with ROGUE Metric

The [ROUGE metric](https://en.wikipedia.org/wiki/ROUGE_(metric)) helps quantify the validity of summarizations produced by models. It compares summarizations to a "baseline" summary which is usually created by a human. While not perfect, it does give an indication to the overall increase in summarization effectiveness that we have accomplished by fine-tuning.

In [17]:
%pip install --disable-pip-version-check -q \
    transformers==4.27.2 \
    datasets==2.9.0 \
    accelerate==0.17.0 \
    promptsource==0.2.3 \
    evaluate==0.4.0 \
    trl==0.4.1 \
    rouge_score==0.1.2 \
    loralib==0.1.1

Note: you may need to restart the kernel to use updated packages.


In [18]:
%pip install git+https://github.com/huggingface/peft.git

  Cloning https://github.com/huggingface/peft.git to /tmp/pip-req-build-alfhg_80
  Running command git clone --filter=blob:none --quiet https://github.com/huggingface/peft.git /tmp/pip-req-build-alfhg_80
  Resolved https://github.com/huggingface/peft.git to commit b21559e042555a8370f4b3cf1ad581c45291cf15
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
Note: you may need to restart the kernel to use updated packages.


In [19]:
%pip install git+https://github.com/lvwerra/trl.git

  Cloning https://github.com/lvwerra/trl.git to /tmp/pip-req-build-2udk1se9
  Running command git clone --filter=blob:none --quiet https://github.com/lvwerra/trl.git /tmp/pip-req-build-2udk1se9
  Resolved https://github.com/lvwerra/trl.git to commit 160d0c9d6cdbc8b247bad4aa8f97f2c57dc4a60f
  Preparing metadata (setup.py) ... done
  Created wheel for trl: filename=trl-0.4.2.dev0-py3-none-any.whl size=54069 sha256=8e2af605de59092126892bf7293ccf00678f55e98d703afc8e5ae3e191557628
  Stored in directory: /tmp/pip-ephem-wheel-cache-f1bwhu97/wheels/ca/6e/f4/b183ecbed483efdcd2041a8021ce7bcb9f7b09c74bff5bb00a
Successfully built trl
  Attempting uninstall: trl
    Found existing installation: trl 0.4.1
    Uninstalling trl-0.4.1:
      Successfully uninstalled trl-0.4.1
Note: you may need to restart the kernel to use updated packages.


In [32]:
import argparse
import csv

import evaluate
import numpy as np
import torch
from datasets import load_dataset
from tqdm import tqdm
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer

# https://huggingface.co/spaces/evaluate-measurement/toxicity
toxicity_evaluator = evaluate.load("ybelkada/toxicity",
                                         "DaNLP/da-electra-hatespeech-detection", 
                                         module_type="measurement")
ds = load_dataset("OxAISH-AL-LLM/wiki_toxic", 
                                split="test")

# parser = argparse.ArgumentParser(description="Evaluate de-toxified models")
# parser.add_argument("--model_type", default="all", type=str, help="Relative path to the source model folder")
# parser.add_argument("--output_file", default="toxicity.csv", type=str, help="Relative path to the source model folder")
# parser.add_argument("--batch_size", default=64, type=int, help="Batch size")
# parser.add_argument("--num_samples", default=400, type=int, help="Number of samples")
# parser.add_argument("--context_length", default=2000, type=int, help="Number of samples")
# parser.add_argument("--max_new_tokens", default=30, type=int, help="Max new tokens for generation")
# args = parser.parse_args()

class Args:
    model_type: str
    output_file: str
    batch_size: float
    num_samples: float
    content_length: float
    max_new_tokens: str

args = Args()    
args.model_type='all'
args.output_file='toxicity.csv'
args.batch_size=64
args.num_samples=400
args.context_length=2000
args.max_new_tokens=30

if args.model_type == "all":
    MODELS_TO_TEST = [
        'google/flan-t5-base',
#        './peft_lora_fine_tune_model_checkpoint',
#        './peft_fine_tuned_with_public_qanda',
        './peft_fine_tuned_with_public_dataset_instructions',        
        './peft_fine_tuned_with_detoxification_rewards',
        # "t5-base",        
        #supervised_fine_tuned_model_checkpoint,
        
    ]
    
pretrained_model_checkpoint='google/flan-t5-base'

NUM_SAMPLES = args.num_samples
BATCH_SIZE = args.batch_size
output_file = args.output_file
max_new_tokens = args.max_new_tokens
context_length = args.context_length
device = torch.cuda.current_device() if torch.cuda.is_available() else "cpu"

# consider only toxic prompts
ds = ds.filter(lambda x: x["label"] == 1)

toxicities = {}

# open a csv file
file = open(f"{output_file}", "w", newline="")
writer = csv.writer(file)
# add first rows
writer.writerow(["model_id", "mean_toxicity", "std_toxicity"])

from peft import PeftModel, PeftConfig
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer

for model_id in tqdm(MODELS_TO_TEST):
    # load base LLM model and tokenizer
#     if 'peft' in model_id:
#         pretrained_model = AutoModelForSeq2SeqLM.from_pretrained(pretrained_model_checkpoint, device_map={"": device}, torch_dtype=torch.bfloat16)  #, device_map="auto")

#         # Load the LoRA/PEFT model
#         model = PeftModel.from_pretrained(pretrained_model, model_id, device_map={"": device}, torch_dtype=torch.bfloat16)
#     else:
    model = AutoModelForSeq2SeqLM.from_pretrained(model_id, device_map={"": device}, torch_dtype=torch.bfloat16)
    
    tokenizer = AutoTokenizer.from_pretrained(model_id)
    tokenizer.pad_token = tokenizer.eos_token
    tokenizer.padding_side = "left"
    input_texts = []

    for i, example in enumerate(ds):
        # set seed
        torch.manual_seed(42)

        input_text = example["comment_text"]
        input_texts.append(input_text[:2000])

        if i > NUM_SAMPLES:
            break

        if (i + 1) % BATCH_SIZE == 0:
            inputs = tokenizer(input_texts, return_tensors="pt", padding=True).to(device)
            inputs.input_ids = inputs.input_ids[:context_length]
            inputs.attention_mask = inputs.attention_mask[:context_length]
            outputs = model.generate(**inputs, do_sample=True, max_new_tokens=max_new_tokens, use_cache=True)
            generated_texts = tokenizer.batch_decode(outputs, skip_special_tokens=True)
            generated_texts = [
                generated_text.replace(input_texts[i], "") for i, generated_text in enumerate(generated_texts)
            ]
            toxicity_score = toxicity_evaluator.compute(predictions=generated_texts)
            input_texts = []

            if model_id not in toxicities:
                toxicities[model_id] = []
            toxicities[model_id].extend(toxicity_score["toxicity"])

    # last batch
    inputs = tokenizer(input_texts, return_tensors="pt", padding=True).to(device)
    outputs = model.generate(**inputs, do_sample=True, max_new_tokens=30)
    generated_texts = tokenizer.batch_decode(outputs, skip_special_tokens=True)
    generated_texts = [generated_text.replace(input_texts[i], "") for i, generated_text in enumerate(generated_texts)]
    toxicity_score = toxicity_evaluator.compute(predictions=generated_texts)
    toxicities[model_id].extend(toxicity_score["toxicity"])

    # compute mean & std using np
    mean = np.mean(toxicities[model_id])
    std = np.std(toxicities[model_id])

    # save to file
    writer.writerow([model_id, mean, std])

    # print
    print(f"Model: {model_id} - Mean: {mean} - Std: {std}")

    model = None
    torch.cuda.empty_cache()

# close file
file.close()

Found cached dataset wiki_toxic (/root/.cache/huggingface/datasets/OxAISH-AL-LLM___wiki_toxic/default/1.0.0/09a67129f85f67f22107b0190f7c32050ef0dce44afeedc6e3e0ab7ab3bd709c)
Loading cached processed dataset at /root/.cache/huggingface/datasets/OxAISH-AL-LLM___wiki_toxic/default/1.0.0/09a67129f85f67f22107b0190f7c32050ef0dce44afeedc6e3e0ab7ab3bd709c/cache-411a66332d7a5b58.arrow
 33%|███▎      | 1/3 [00:26<00:52, 26.44s/it]

Model: google/flan-t5-base - Mean: 0.10735442655624947 - Std: 0.2648808616377917


OSError: ./peft_fine_tuned_with_public_dataset_instructions does not appear to have a file named config.json. Checkout 'https://huggingface.co/./peft_fine_tuned_with_public_dataset_instructions/None' for available files.

# Release Resources

In [ ]:
# %%html

# <p><b>Shutting down your kernel for this notebook to release resources.</b></p>
# <button class="sm-command-button" data-commandlinker-command="kernelmenu:shutdown" style="display:none;">Shutdown Kernel</button>
        
# <script>
# try {
#     els = document.getElementsByClassName("sm-command-button");
#     els[0].click();
# }
# catch(err) {
#     // NoOp
# }    
# </script>